# Face recognition using LBPH recognizer algorithm

## Importing libraries

In [1]:
import os
import time
import uuid
import cv2
import pandas as pd
import numpy as np
import datetime
import time
from PIL import Image,ImageTk

In [ ]:
# Image ID
ID = int(input("Enter the ID:: "))

# Name
Name = input("Enter the Name:: ")

Enter the ID:: 0


In [3]:
# Dataset Path
IMAGE_PATH = "/home/christ-infotech-007/PycharmProjects/pythonProject6/dataset"

In [4]:
#Turning on the live stream using web cam (-1 for LINUX, 0 for WINDOW, and 1 for MAC-OS)

source = cv2.VideoCapture(-1)
detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [5]:
# Sample Size
sampleNum=300

#Image Number
frame_num = 0

In [6]:
# running the loop
for sample in range(sampleNum):
  
    # extracting the frames
    ret, img = source.read()
      
    # converting to gray-scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = detector.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        
        #Increament the image number
        frame_num+=1
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        imgname = os.path.join(IMAGE_PATH,Name+"."+str(ID)+"."+f'{str(frame_num)}.jpg')
        cv2.imwrite(imgname, gray)
        # displaying the video
        cv2.imshow("Frame", gray)
    
    
      
# closing the window
cv2.destroyAllWindows()
source.release()

## Training the model

In [7]:
# Define function to create images and labels for training

def getImagesAndLabels(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    # create empth face list
    faceSamples = []
    # create empty ID list
    Ids = []
    # now looping through all the image paths and loading the Ids and the images
    for imagePath in imagePaths:
        # loading the image and converting it to gray scale
        pilImage = Image.open(imagePath).convert('L')
        # Now we are converting the PIL image into numpy array
        imageNp = np.array(pilImage, 'uint8')
        # getting the Id from the image

        Id = int(os.path.split(imagePath)[-1].split(".")[1])
        # extract the face from the training image sample
        faces = detector.detectMultiScale(imageNp)
        # If a face is there then append that in the list as well as Id of it
        for (x, y, w, h) in faces:
            faceSamples.append(imageNp[y:y + h, x:x + w])
            Ids.append(Id)
    return faceSamples, Ids

In [8]:
# recognizing face

recognizer = cv2.face.LBPHFaceRecognizer_create()

In [9]:
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [10]:
image_path = "/home/christ-infotech-007/PycharmProjects/pythonProject6/dataset"
faces, Id = getImagesAndLabels(image_path)

In [11]:
# Training the model
recognizer.train(faces, np.array(Id))

In [12]:
# Saving the model
recognizer.save("model/trained_model2.yml")

## Testing the model with web cam

In [13]:
#loading the model

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('model/trained_model2.yml')
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
font = cv2.FONT_HERSHEY_SIMPLEX
id = 0

# add the list of names of your dataset here
names = ['rohan'] 


cam = cv2.VideoCapture(-1)

while True:
    ret, img =cam.read()
    #img = cv2.flip(img, -1) # Flip vertically
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5
       )
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        
        # If confidence is less them 100 ==> "0" : perfect match 
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(
                    img, 
                    str(id), 
                    (x+5,y-5), 
                    font, 
                    1, 
                    (255,255,255), 
                    2
                   )
         
    
    cv2.imshow('camera',img) 
    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 